In [1]:
with open("input.txt", "r") as file:
    content = file.readlines()

In [2]:
def str_clean(sent):

    append_to_prev = False

    if len(sent) > 0 and not sent[0] =='[':
        append_to_prev = True
        sent = sent[:-1]
        return append_to_prev, sent

    else:
        sent = sent.split(":")[2:]

        if len(sent) == 0 or sent[-1].isspace():
            return None, None
        
        sent[-1] = sent[-1][:-1]
        sent = ":".join(sent)
        return append_to_prev, sent


content_clean = []
clean_index = []
for i, c in enumerate(content):
    append_to_prev, sent = str_clean(c)
    if sent is not None:
        if append_to_prev:
            content_clean[-1] += ' \n' + sent
        else:
            content_clean.append(sent)
            clean_index.append(i)



In [ ]:
# Test block

In [2]:
from transformers import pipeline
# from transformers import AutoTokenizer, AutoModelForTokenClassification


### For ner
# tokenizer = AutoTokenizer.from_pretrained("dslim/bert-base-NER")
# model = AutoModelForTokenClassification.from_pretrained("dslim/bert-base-NER")
# ner_nlp = pipeline("ner", model=model, tokenizer=tokenizer)
ner_nlp = pipeline("ner")



### For query_llm 
# (login to HF using 'huggingface-cli login --token $HUGGINGFACE_TOKEN' before this)
# model_llm = "meta-llama/Llama-2-7b-chat-hf"
# tokenizer = AutoTokenizer.from_pretrained(model_llm)
# chat_llm = pipeline(
#     "text-generation",
#     model=model_llm,
#    # torch_dtype=torch.float16,
#     device_map="auto",
# )
chat_llm = pipeline("text-generation")



### For chain_of_density_summarise (we're using the regular summariser for now)
summarizer = pipeline("summarization")

/home/stoch/.local/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
No model was supplied, defaulted to dbmdz/bert-large-cased-finetuned-conll03-english and revision f2482bf (https://huggingface.co/dbmdz/bert-large-cased-finetuned-conll03-english).
Using a pipeline without specifying a model name and revision in production is not recommended.
Some weights of the model checkpoint at dbmdz/bert-large-cased-finetuned-conll03-english were not used when initializing BertForTokenClassification: ['bert.pooler.dense.weight', 'bert.pooler.dense.bias']
- This IS expected if you are initializing BertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expecte

In [9]:
import os
import tweepy

consumer_key = os.environ["X_CONSUMER_KEY"] 
consumer_secret = os.environ["X_CONSUMER_SECRET"]
access_key = os.environ["X_ACCESS_KEY"]
access_secret = os.environ["X_ACCESS_SECRET"]

# auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
# auth.set_access_token(access_key, access_secret)

# # Create API object
# api = tweepy.API(auth)

oauth2_user_handler = tweepy.OAuth2UserHandler(
    client_id=consumer_key,
    redirect_uri="https://localhost/",
    scope=["tweet.read"],
    # Client Secret is only necessary if using a confidential client
    client_secret=consumer_secret
)

access_token = oauth2_user_handler.fetch_token(
    "Authorization Response URL here"
)

client = tweepy.Client("Access Token here")



# client = tweepy.Client(
#     consumer_key=consumer_key,
#     consumer_secret=consumer_secret,
#     access_token=access_key,
#     access_token_secret=access_secret
# )



def get_from_x(url):
    # Get tweet ID from tweet URL
    tweet_url = 'https://twitter.com/knit_ai/status/1735014242808647845?s=20'
    tweet_split = tweet_url.split('/')
    
    found = False
    for i, t in enumerate(tweet_split):
        if t == 'status':
            found = True
            break
    
    if not found:
        return ''
    
    tweet_id = tweet_url[i+1]
    # tweet_user_id = tweet_url[i-1]

    # Get tweet ID using the Twitter API
    tweet = client.get_tweet(tweet_id)
    print(tweet)
    # Print tweet ID
    # print(f"Tweet: {tweet.text}")


get_from_x('x')


InsecureTransportError: (insecure_transport) OAuth 2 MUST utilize https.

In [32]:
import re
from urllib.request import urlopen
from bs4 import BeautifulSoup



def get_context_from_links(text):
    context = ''
    urls = re.findall('http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+', text)
    for url in urls:

        if 'x.com' in url or 'twitter.com' in url:
            context += '\n' + get_from_x(url)
        
        else:

            html = urlopen(url).read()
            soup = BeautifulSoup(html, features="html.parser")

            # kill all script and style elements
            for script in soup(["script", "style"]):
                script.extract()    # rip it out

            # get text
            text = soup.get_text()

            # break into lines and remove leading and trailing space on each
            lines = (line.strip() for line in text.splitlines())
            # break multi-headlines into a line each
            chunks = (phrase.strip() for line in lines for phrase in line.split("  "))
            # drop blank lines
            text = '\n'.join(chunk for chunk in chunks if chunk)

            context += '\n {}'.format(text)

    if context != '':
        return chain_of_density_summarise(context, '', min(len(context), 300))
    else:
        return ''



def chain_of_density_summarise(prev, next, max_length=300):
    # Regular summariser for now
    summarized = summarizer(prev + next, min_length=0, max_length=max_length)
    return summarized[0]['summary_text']



def query_llm(prompt):
    sequences = chat_llm(
        prompt,
        do_sample=True,
        top_k=10,
        num_return_sequences=1,
        # eos_token_id=tokenizer.eos_token_id,
        max_length=200,
    )
    
    return sequences[0]['generated_text']

In [34]:
summaries = []

print('total entries: {}'.format(len(content_clean)))

# for i in range(len(content_clean)):
# for i in range(10):
for i in [2]:

    print('working on entry {}'.format(i))

    ner_content = ner_nlp(content_clean[i])
    overall_context = content_clean[i]

    for d in ner_content:
        prompt = "What is {} ? \nContext: {}. Keep it very brief.".format(d['word'], content_clean[i])
        response = query_llm(prompt)
        overall_context += '\n' + response

    context_from_links = get_context_from_links(content_clean[i])
    overall_context += "\n\n" + context_from_links
        

    added = False
    for s in summaries:
        prompt = "Are these things part of the same conversation ? {} and {}. Strictly reply in 'yes' or 'no'.".format(s['content'], overall_context)
        response = query_llm(prompt)

        if 'yes' in response:
            s['content'] = chain_of_density_summarise(s['content'], overall_context)
            s['index'].append(clean_index[i])
            added = True
        else:
            pass

    if added is False:
        s_new = {
            'content': chain_of_density_summarise('', overall_context),
            'index': [clean_index[i]]
        }
        summaries.append(s_new)


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


total entries: 254
working on entry 2


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


HTTPError: HTTP Error 403: Forbidden

In [35]:
# summaries
content_clean[2]

' Found something interesting when integrating Google Gemini on our platform. Just ask it "Give your thoughts on openai". Full details https://x.com/knit_ai/status/1735014242808647845?s=20'

In [36]:
### Testing

def get_context_from_links2(text):
    context = ''
    urls = re.findall('http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+', text)
    for url in urls:
        html = urlopen(url).read()
        soup = BeautifulSoup(html, features="html.parser")

        # kill all script and style elements
        for script in soup(["script", "style"]):
            script.extract()    # rip it out

        # get text
        text = soup.get_text()

        # break into lines and remove leading and trailing space on each
        lines = (line.strip() for line in text.splitlines())
        # break multi-headlines into a line each
        chunks = (phrase.strip() for line in lines for phrase in line.split("  "))
        # drop blank lines
        text = '\n'.join(chunk for chunk in chunks if chunk)

        context += '\n {}'.format(text)
    
    # print("\n\n")
    # print(context)
    if context != '':
        return chain_of_density_summarise(context, '')
    else:
        return ''



i = 2

print('working on entry {}'.format(i))

ner_content = ner_nlp(content_clean[i])
overall_context = content_clean[i]

for d in ner_content:
    prompt = "What is {} ? \nContext: {}. Keep it very brief.".format(d['word'], content_clean[i])
    response = query_llm(prompt)
    overall_context += '\n' + response

context_from_links = get_context_from_links2(content_clean[i])[0]['summary_text']
overall_context += "\n\n" + context_from_links

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


working on entry 2


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


HTTPError: HTTP Error 403: Forbidden